In [1]:
from fastapi import FastAPI, Request, status
from fastapi.responses import Response, StreamingResponse
import nest_asyncio
nest_asyncio.apply()
import uvicorn
from pydantic import BaseModel
from dotenv import load_dotenv
load_dotenv()
import openai
import os
import sys
from pathlib import Path
from openai import OpenAI
import ast
import json
from fastapi.middleware.cors import CORSMiddleware

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not len(OPENAI_API_KEY):
    print("Set OPENAI_API_KEY")


In [4]:
class Speech(BaseModel):
    excerpt:str
    previous_suggestion:str
    topic: str

app = FastAPI()

origins = [
    "http://localhost:4321",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials = True,
    allow_methods=["*"],
    allow_headers=["*"]
)




In [5]:
def generate_speech_async(excerpt, previous_suggestion,topic):
    prompt = [{"role":"system","content":'''Your job is to continue my speech in 1 short sentence. Do not say anything else. 
    Your response should be string. I read your suggestions, but I can choose not to follow them. If I am not yet done
    reading your previous suggestion, output your previous suggestion. Else, output a new sentence to continue my speech.'''},
        {"role":"user","content":f"Topic: {topic}"},
        {"role":"user","content":f"Speech so far: {excerpt}"},
        {"role":"user","content":f"Previous suggestion: {previous_suggestion}"}
    ]
    return prompt

async def standard_stream_generator(response, **kwargs):
    msg = ""
    for chunk in response:
        try:
            chunk_response = chunk.choices[0].delta.content
            msg += chunk_response
            print(msg)
            yield msg.encode()
        except:
            yield ""
            continue

@app.post("/generate_speech_async",status_code=status.HTTP_202_ACCEPTED)
async def continue_speech_async(speech:Speech):
    messages = generate_speech_async(excerpt = speech.excerpt, previous_suggestion = speech.previous_suggestion, topic = speech.topic)
    print(messages)
    try:
        client = OpenAI()
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            temperature = 0.7,
            messages = messages,
            stream = True
        )
        return StreamingResponse(standard_stream_generator(response,excerpt = speech.excerpt, previous_suggestion = speech.previous_suggestion,
                                    topic = speech.topic),
                                    media_type='text/event-stream')

    except Exception as e:
        print(e)
        return ""
uvicorn.run(app,host="192.168.68.126",port=8080)

INFO:     Started server process [15660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://192.168.68.126:8080 (Press CTRL+C to quit)


INFO:     192.168.68.126:62232 - "OPTIONS /generate_speech_async HTTP/1.1" 200 OK
[{'role': 'system', 'content': 'Your job is to continue my speech in 1 short sentence. Do not say anything else. \n    Your response should be string. I read your suggestions, but I can choose not to follow them. If I am not yet done\n    reading your previous suggestion, output your previous suggestion. Else, output a new sentence to continue my speech.'}, {'role': 'user', 'content': 'Topic: '}, {'role': 'user', 'content': 'Speech so far: good morning'}, {'role': 'user', 'content': 'Previous suggestion: '}]
INFO:     192.168.68.126:62232 - "POST /generate_speech_async HTTP/1.1" 200 OK

Good
Good morning
Good morning,
Good morning, ladies
Good morning, ladies and
Good morning, ladies and gentlemen
Good morning, ladies and gentlemen.
[{'role': 'system', 'content': 'Your job is to continue my speech in 1 short sentence. Do not say anything else. \n    Your response should be string. I read your suggestions,

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [15660]
